In [1]:
import pyspark as spark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id
import re
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [3]:
df1=pd.read_csv('/Users/harishpuvvada/Desktop/PBDA/IPynb Spark/tweetsfinal.csv',error_bad_lines=False,engine = 'python',header = None) 
df2=pd.read_csv('/Users/harishpuvvada/Desktop/PBDA/IPynb Spark/BitCoinPrice.csv',error_bad_lines=False,engine = 'python',header = None) 
FullDataTw=sql.createDataFrame(df1)
FullDataBtc=sql.createDataFrame(df2) #creating pandas df and then changing it to pyspark df

Skipping line 845142: unexpected end of data


In [4]:
FullDataTw = FullDataTw.dropna() #getting rid of full empty rows
print(FullDataTw.count())
print(FullDataBtc.count())

845141
217


In [5]:
FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")
FullDataTw = FullDataTw.withColumnRenamed('0', 'DateTime') #setting column names of Twitter dataset
FullDataTw = FullDataTw.withColumnRenamed('1', 'Tweet')
FullDataBtc = FullDataBtc.withColumnRenamed('0', 'DateTime') #setting column names of Bitcoin price dataset
FullDataBtc = FullDataBtc.withColumnRenamed('1', 'Price')
FullDataBtc = FullDataBtc.filter(FullDataBtc.DateTime != 'Date') #to get rid of first row with the header

In [6]:
Tw_samp = FullDataTw.limit(50) #taking sample of 50 rows and working on it

In [7]:
import preprocessor as p #cleaning each tweet using tweet-preprocessor like removing hashtags,urls,emojis....
def function_udf(input_str):
    input_str = re.sub(r'RT', '', input_str)
    p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
    input_str = p.clean(input_str)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())
func_udf = udf(function_udf, StringType())
CleanDF = Tw_samp.withColumn('CleanedTweets', func_udf(Tw_samp['Tweet']))
CleanDF.show(5)

+--------------------+--------------------+--------------------+
|            DateTime|               Tweet|       CleanedTweets|
+--------------------+--------------------+--------------------+
|Thu Nov 09 17:43:...|RT @Forbes: The F...|The Failure of Se...|
|Thu Nov 09 17:43:...|RT @mindstatex: L...|Lots of love from...|
|Thu Nov 09 17:43:...|RT @FernandoHuama...|Warning Built in ...|
|Thu Nov 09 17:43:...|RT @LevelNetwork:...|Join our telegram...|
|Thu Nov 09 17:43:...|RT @realsheepwolf...|DIGAF FLOAT 16M T...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [8]:
from textblob import TextBlob  #passing cleaned tweets and getting a sentiment score for each tweet
def senti_score_udf(input_str):
    analysis = TextBlob(input_str)
    return analysis.sentiment.polarity
func_udf2 = udf(senti_score_udf, FloatType())
CleanDF = CleanDF.withColumn('Sentiment_score', func_udf2(CleanDF['CleanedTweets']))
CleanDF.show(5)

+--------------------+--------------------+--------------------+---------------+
|            DateTime|               Tweet|       CleanedTweets|Sentiment_score|
+--------------------+--------------------+--------------------+---------------+
|Thu Nov 09 17:43:...|RT @Forbes: The F...|The Failure of Se...|    -0.18888889|
|Thu Nov 09 17:43:...|RT @mindstatex: L...|Lots of love from...|     0.25833333|
|Thu Nov 09 17:43:...|RT @FernandoHuama...|Warning Built in ...|            0.0|
|Thu Nov 09 17:43:...|RT @LevelNetwork:...|Join our telegram...|            0.0|
|Thu Nov 09 17:43:...|RT @realsheepwolf...|DIGAF FLOAT 16M T...|          -0.05|
+--------------------+--------------------+--------------------+---------------+
only showing top 5 rows



In [9]:
def Tw_Time_format(stri):  #manipulating and casting the strings(DateTime) of tweets dataframe to timestamps
    dic = {'Nov':'11','Oct':'10'}
    ans = ''
    ans += stri[-4:]+'-'+ dic[stri[4:7]]+'-'+stri[8:19]
    return ans
func_udf3 = udf(Tw_Time_format,StringType())
CleanDF = CleanDF.withColumn('DateTime_c', func_udf3(CleanDF['DateTime']))
CleanDF = CleanDF.withColumn("DateTime_casted",CleanDF['DateTime_c'].cast(TimestampType()))
CleanDF.show(5)

+--------------------+--------------------+--------------------+---------------+-------------------+-------------------+
|            DateTime|               Tweet|       CleanedTweets|Sentiment_score|         DateTime_c|    DateTime_casted|
+--------------------+--------------------+--------------------+---------------+-------------------+-------------------+
|Thu Nov 09 17:43:...|RT @Forbes: The F...|The Failure of Se...|    -0.18888889|2017-11-09 17:43:41|2017-11-09 17:43:41|
|Thu Nov 09 17:43:...|RT @mindstatex: L...|Lots of love from...|     0.25833333|2017-11-09 17:43:40|2017-11-09 17:43:40|
|Thu Nov 09 17:43:...|RT @FernandoHuama...|Warning Built in ...|            0.0|2017-11-09 17:43:39|2017-11-09 17:43:39|
|Thu Nov 09 17:43:...|RT @LevelNetwork:...|Join our telegram...|            0.0|2017-11-09 17:43:39|2017-11-09 17:43:39|
|Thu Nov 09 17:43:...|RT @realsheepwolf...|DIGAF FLOAT 16M T...|          -0.05|2017-11-09 17:43:39|2017-11-09 17:43:39|
+--------------------+----------

In [10]:
FinalTw = CleanDF.selectExpr("DateTime_casted as Date_Time", "CleanedTweets as Cleaned_Tweets","Sentiment_score")
FinalTw.show(5) #selecting necessary columns

+-------------------+--------------------+---------------+
|          Date_Time|      Cleaned_Tweets|Sentiment_score|
+-------------------+--------------------+---------------+
|2017-11-09 17:43:41|The Failure of Se...|    -0.18888889|
|2017-11-09 17:43:40|Lots of love from...|     0.25833333|
|2017-11-09 17:43:39|Warning Built in ...|            0.0|
|2017-11-09 17:43:39|Join our telegram...|            0.0|
|2017-11-09 17:43:39|DIGAF FLOAT 16M T...|          -0.05|
+-------------------+--------------------+---------------+
only showing top 5 rows



In [11]:
FinalTw.printSchema()

root
 |-- Date_Time: timestamp (nullable = true)
 |-- Cleaned_Tweets: string (nullable = true)
 |-- Sentiment_score: float (nullable = true)



In [12]:
def Btc_Time_format(input_str): #manipulating and casting the strings(DateTime) of BTC dataframe to timestamps
    input_str = '2017-'+ input_str
    input_str = re.sub(r'/', '-', input_str)
    input_str += ':00'
    return input_str[:10]+input_str[13:]
func_udf = udf(Btc_Time_format, StringType())
FullDataBtc = FullDataBtc.withColumn('Cleaned_BTC_Time', func_udf(FullDataBtc['DateTime']))

In [13]:
CleandfBtc = FullDataBtc.withColumn("Cleaned_BTC_Time_New",FullDataBtc['Cleaned_BTC_Time'].cast(TimestampType()))
FinalBtc = CleandfBtc.selectExpr("Cleaned_BTC_Time_New as Date_Time", "Price")
FinalBtc = FinalBtc.withColumn("Price",FinalBtc['Price'].cast(DoubleType()))
FinalBtc.show(5)#In this cell, casting to timesstamp, changing col names and casting price type to double

+-------------------+-------+
|          Date_Time|  Price|
+-------------------+-------+
|2017-10-31 00:00:00|6142.46|
|2017-10-31 01:00:00|6139.47|
|2017-10-31 02:00:00| 6128.2|
|2017-10-31 03:00:00|6130.72|
|2017-10-31 04:00:00|6143.92|
+-------------------+-------+
only showing top 5 rows



In [14]:
FinalBtc.printSchema()

root
 |-- Date_Time: timestamp (nullable = true)
 |-- Price: double (nullable = true)



In [15]:
# dates_df.registerTempTable("dates")
# events_df.registerTempTable("events")
# results = sqlContext.sql("SELECT * FROM dates INNER JOIN events ON dates.lower_timestamp < events.time and  events.time < dates.upper_timestamp")
# results.explain()